<font size="6">Create starting non-soil carbon density rasters from WHRC AGB 2000: aboveground carbon, belowground carbon, deadwood carbonn, litter carbon</font> 

<font size="4">Must be run using the utilities_and_variables.ipynb kernel</font> 

In [77]:
# Function to create initial non-soil carbon pool densities
# Operates pixel by pixel, so uses numba (Python compiled to C++).
@jit(nopython=True)
def create_starting_C_densities(in_dict_uint8, in_dict_int16, in_dict_float32):

    # # Output array of 0s
    # IPCC_change_block = np.zeros(IPCC_previous_block.shape)

    # # Iterates through all pixels in the chunk
    # for row in range(IPCC_previous_block.shape[0]):
    #     for col in range(IPCC_previous_block.shape[1]):

    #         IPCC_previous = IPCC_previous_block[row, col]
    #         IPCC_current = IPCC_current_block[row, col]

    #         # When land cover chunks have "no data"
    #         if (IPCC_previous == 0) and (IPCC_current == 0):
    #             IPCC_change_block[row, col] = 0

    #         else:
    #             # Equation to calculate the IPCC change code
    #             IPCC_change_block[row, col] = ((IPCC_previous - 1) * IPCC_class_max_val) + IPCC_current

    # return IPCC_change_block
    return

In [95]:
# 
def create_and_upload_starting_C_densities(bounds, is_final):

    bounds_str = boundstr(bounds)    # String form of chunk bounds
    tile_id = xy_to_tile_id(bounds[0], bounds[3])    # tile_id in YYN/S_XXXE/W
    chunk_length_pixels = calc_chunk_length_pixels(bounds)   # Chunk length in pixels (as opposed to decimal degrees)

    no_data_val = 255

    
    ### Part 1: download chunks and check for data

    # Dictionary of downloaded layers
    layers = {}

    download_dict = {
        
        agb_2000: f"s3://gfw2-data/climate/WHRC_biomass/WHRC_V4/Processed/{tile_id}_t_aboveground_biomass_ha_2000.tif",
        elevation: f"s3://gfw2-data/climate/carbon_model/inputs_for_carbon_pools/processed/elevation/20190418/{tile_id}_elevation.tif",
        climate_domain: f"s3://gfw2-data/climate/carbon_model/inputs_for_carbon_pools/processed/fao_ecozones_bor_tem_tro/20190418/{tile_id}_fao_ecozones_bor_tem_tro_processed.tif",
        precipitation: f"s3://gfw2-data/climate/carbon_model/inputs_for_carbon_pools/processed/precip/20190418/{tile_id}_precip_mm_annual.tif"
    }

    # Checks whether tile exists at all. Doesn't try to download chunk if the tile doesn't exist.
    tile_exists = check_for_tile(download_dict, is_final)

    if tile_exists == 0:
        return

    futures = prepare_to_download_chunk(bounds, download_dict, no_data_val)

    if not is_final:
        print(f"Waiting for requests for data in chunk {bounds_str} in {tile_id}: {timestr()}")
    
    # Waits for requests to come back with data from S3
    for future in concurrent.futures.as_completed(futures):
        layer = futures[future]
        layers[layer] = future.result()

    # Checks chunk for data. Skips the chunk if it has no data in it.
    data_in_chunk = check_chunk_for_data(layers, agb_2000, bounds_str, tile_id, no_data_val, is_final)

    if data_in_chunk == 0:
        return
        
        
    ### Part 2: Create a separate dictionary for each chunk datatype so that they can be passed to Numba as separate arguments.
    ### Numba functions can accept (and return) dictionaries of arrays as long as each dictionary only has arrays of one data type (e.g., uint8, float32)
    ### Note: need to add new code if inputs with other data types are added

    typed_dict_uint8, typed_dict_int16, typed_dict_int32, typed_dict_float32 = create_typed_dicts(layers)

    # # Initializes empty dictionaries for each type
    # uint8_dict_layers = {}
    # int16_dict_layers = {}
    # int32_dict_layers = {}
    # float32_dict_layers = {}
    
    # # Iterates through the downloaded chunk dictionary and distributes arrays to a separate dictionary for each data type
    # for key, array in layers.items():
    #     if array.dtype == np.uint8:
    #         uint8_dict_layers[key] = array
    #     elif array.dtype == np.int16:
    #         int16_dict_layers[key] = array
    #     elif array.dtype == np.int32:
    #         int32_dict_layers[key] = array
    #     elif array.dtype == np.float32:
    #         float32_dict_layers[key] = array
    #     else:
    #         raise TypeError(f"{key} dtype not in list")

    # print(f"uint8 datasets: {uint8_dict_layers.keys()}")
    # print(f"int16 datasets: {int16_dict_layers.keys()}")
    # print(f"int32 datasets: {int32_dict_layers.keys()}")
    # print(f"float32 datasets: {float32_dict_layers.keys()}")
    
    # # Creates numba-compliant typed dict for each type of array
    # typed_dict_uint8 = Dict.empty(
    #     key_type=types.unicode_type, 
    #     value_type=types.Array(types.uint8, 2, 'C')  # Assuming 2D arrays of uint8
    # )

    # typed_dict_int16 = Dict.empty(
    #     key_type=types.unicode_type, 
    #     value_type=types.Array(types.int16, 2, 'C')  # Assuming 2D arrays of int16
    # )
    
    # typed_dict_int32 = Dict.empty(
    #     key_type=types.unicode_type, 
    #     value_type=types.Array(types.int32, 2, 'C')  # Assuming 2D arrays of int32
    # )
    
    # typed_dict_float32 = Dict.empty(
    #     key_type=types.unicode_type, 
    #     value_type=types.Array(types.float32, 2, 'C')  # Assuming 2D arrays of float32
    # )

    # # Populates the numba-compliant typed dicts
    # for key, array in uint8_dict_layers.items():
    #     typed_dict_uint8[key] = array

    # for key, array in int16_dict_layers.items():
    #     typed_dict_int16[key] = array

    # for key, array in int32_dict_layers.items():
    #     typed_dict_int32[key] = array

    # for key, array in float32_dict_layers.items():
    #     typed_dict_float32[key] = array
    
    
    # ### Part 3: Create starting carbon pool densities and upload them to s3

    # print(f"Creating starting C densities for {bounds_str} in {tile_id} for {year}: {timestr()}")

    # # Create AGC, BGC, deadwood C and litter C
    # IPCC_classes = create_starting_C_densities(
    #     typed_dict_uint8, typed_dict_int16, typed_dict_float32  
    # )

    # # # Output arrays to upload to s3. Adds new array to dictionary for each year
    # # IPCC_class_dict[f"{IPCC_class_pattern}_{year}"] = [IPCC_classes, IPCC_classes.dtype.name, IPCC_class_path, year]                 
    
    # # print(IPCC_class_dict)
    
    # # save_and_upload_small_raster_set(bounds, chunk_length_pixels, tile_id, bounds_str, IPCC_class_dict, is_final)

    
    # # # Clear memory of unneeded arrays
    # # del IPCC_classes
    # # del IPCC_class_dict
    # # del IPCC_change
    # # del IPCC_change_dict

    # return f"Success for {bounds_str}: {timestr()}"

In [96]:
%%time

## Create LULUCF flux and carbon stock 2x2 deg rasters 

## Area to analyze
## chunk_params arguments: W, S, E, N, chunk size (degrees)
# chunk_params = [-180, -60, 180, 80, 2]  # entire world
# chunk_params = [-10, 40, 20, 70, 1]    # 30x30 deg (70N_010W), 900 chunks

# chunk_params = [-10, 60, 0, 70, 1]    # 10x10 deg (70N_010W), 100 chunks
# chunk_params = [-10, 65, -5, 70, 1]    # 5x5 deg (70N_010W), 25 chunks
# chunk_params = [-10, 68, -8, 70, 1]    # 2x2 deg (70N_010W), 4 chunks
# chunk_params = [-10, 69, -9, 70, 1]    # 1x1 deg (70N_010W), 1 chunk

# chunk_params = [10, 40, 20, 50, 2]    # 10x10 deg (50N_010E), 25 chunks
# chunk_params = [10, 40, 20, 50, 10]    # 10x10 deg (50N_010E), 1 chunk
# chunk_params = [10, 46, 14, 50, 2]   # 4x4 deg, 4 chunks
# chunk_params = [110, -10, 114, -6, 2]   # 4x4 deg, 4 chunks
# chunk_params = [10, 48, 12, 50, 1]   # 2x2 deg, 4 chunks
# chunk_params = [10, 49, 11, 50, 1]   # 1x1 deg, 1 chunk
# chunk_params = [10, 49, 11, 50, 0.5] # 1x1 deg, 4 chunks
# chunk_params = [10, 49.5, 10.5, 50, 0.25] # 0.5x0.5 deg, 4 chunks
# chunk_params = [10, 42, 11, 43, 0.5] # 1x1 deg, 4 chunks (some GLCLU code=254 for ocean and some land, so data should be output)
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data, no fire)
chunk_params = [15, 41.75, 15.25, 42, 0.25] # 0.25x0.25 deg, 1 chunk (has data with fire)

# # Range of no-data cases for testing
# chunk_params = [20, 69.75, 20.25, 70, 0.25] # 0.25x0.25 deg, 1 chunk (tile exists for GLCLU but not all other inputs, e.g., fire)
# chunk_params = [110, -10, 120, 0, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [110, -20, 120, -10, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [0, 79.75, 0.25, 80, 0.25] # 0.25x0.25 deg, 1 chunk (no 80N_000E tile-- no data)
# chunk_params = [112, -12, 116, -8, 2]   # 2x2 deg, 1 chunk (bottom of Java, has data but mostly ocean)
# chunk_params = [10.875, 41.75, 11, 42, 0.25] # 0.25x0.25 deg, 1 chunk (entirely GLCLU code=255 for ocean, so no actual data-- nothing should be be output)
# chunk_params = [-10, 21.75, -9.75, 22, 0.25] # 0.25x0.25 deg, 1 chunk (has data but entirely desert (fully GLCLU code=0))
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data)


# Makes list of chunks to analyze
chunks = get_chunk_bounds(chunk_params)  
print("Processing", len(chunks), "chunks")
# print(chunks)

# Determines if the output file names for final versions of outputs should be used
is_final = False
if len(chunks) > 90:
    is_final = True
    print("Running as final model.")

# Creates list of tasks to run (1 task = 1 chunk for all years)
delayed_result = [dask.delayed(create_and_upload_starting_C_densities)(chunk, is_final) for chunk in chunks]

# Actually runs analysis
results = dask.compute(*delayed_result)
results

Processing 1 chunks
Tile id _ha_2000 exists. Proceeding.
Requesting data in chunk 15_42_15_42 in 50N_010E: 20240712_21_22_58
Waiting for requests for data in chunk 15_42_15_42 in 50N_010E: 20240712_21_23_00
Data in chunk 15_42_15_42. Proceeding.
uint8 datasets: dict_keys([])
int16 datasets: dict_keys(['elevation', 'agb_2000', 'fao_ecozone_climate_domain'])
int32 datasets: dict_keys(['precipitation_annual'])
float32 datasets: dict_keys([])
CPU times: user 40.3 ms, sys: 1.53 ms, total: 41.8 ms
Wall time: 2.64 s


(None,)